In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/sample_sales.csv")
df.head()

In [ ]:
df["revenue"] = df["price"] * df["units_sold"]
df["gross_profit"] = (df["price"] - df["unit_cost"]) * df["units_sold"]
df["cm_pct"] = (df["price"] - df["unit_cost"]) / df["price"]

sku_metrics = df.sort_values("gross_profit", ascending=False)
sku_metrics[["sku","category","price","unit_cost","units_sold","revenue","gross_profit","cm_pct"]].head(10)


In [ ]:
elasticity_map = {
    "Snacks": -1.2,
    "Drinks": -0.8,
    "Household": -0.5,
    "PersonalCare": -0.9
}

df["elasticity"] = df["category"].map(elasticity_map)
df[["sku","category","price","units_sold","elasticity"]].head()


In [ ]:
def run_scenario(price_change_pct: float) -> pd.DataFrame:
    x = df.copy()

    x["new_price"] = x["price"] * (1 + price_change_pct)
    x["volume_change_pct"] = x["elasticity"] * price_change_pct
    x["new_units"] = x["units_sold"] * (1 + x["volume_change_pct"])

    x["new_revenue"] = x["new_price"] * x["new_units"]
    x["new_profit"] = (x["new_price"] - x["unit_cost"]) * x["new_units"]

    summary = pd.DataFrame({
        "scenario": [f"Price {price_change_pct:+.0%}"],
        "revenue": [x["new_revenue"].sum()],
        "profit": [x["new_profit"].sum()],
        "units": [x["new_units"].sum()]
    })
    return summary

base_revenue = df["revenue"].sum()
base_profit = df["gross_profit"].sum()
base_units = df["units_sold"].sum()

scenarios = pd.concat([
    run_scenario(-0.05),
    run_scenario(0.00),
    run_scenario(0.05),
    run_scenario(0.10),
], ignore_index=True)

scenarios["delta_profit"] = scenarios["profit"] - base_profit
scenarios["delta_profit_pct"] = scenarios["delta_profit"] / base_profit

scenarios["delta_revenue"] = scenarios["revenue"] - base_revenue
scenarios["delta_revenue_pct"] = scenarios["delta_revenue"] / base_revenue

scenarios
